# Connecting Locode info with exisitng E-76 data

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import * 

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

/tmp/ipykernel_967/1862119733.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


## Importing Data
* using data from read in GCS:
    * [E-76 Obligated List](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)
    * Obtained Locode and Agency list 

In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False).drop('Unnamed: 0', axis=1)

In [3]:
df.head()

prefix project_no             agency prepared_date  fed_requested  \
0  BPMPL  5904(121)    Humboldt County    2018-12-18           0.00   
1     ER  32D0(008)   Mendocino County    2018-12-17       11508.00   
2     ER  4820(004)    Humboldt County    2018-12-07       45499.64   
3    CML  5924(244)  Sacramento County    2018-12-11      207002.00   
4    CML  5924(214)  Sacramento County    2018-12-05           0.00   

   ac_requested  total_requested status_comment locode  dist   ftip_no  \
0           0.0             0.00     Authorized   5904     1  HBPLOCAL   
1           0.0         13000.00     Authorized   5910     1       NaN   
2           0.0         51394.58     Authorized   5904     1       NaN   
3           0.0        247002.00     Authorized   5924     3  SAC25086   
4     5680921.0       5702041.00     Authorized   5924     3  SAC24753   

                                    project_location  \
0                      14 Bridges In Humboldt County   
1               Comptche Ukiah Road, Cr 223 Pm 17.25   
2                                Mattole Rd Pm 43.17   
3     Fair Oaks Blvd. Between Howe Ave And Munroe St   
4  Florin Rd Between Power Inn Rd. And Florin Per...   

                                        type_of_work  seq      mpo  prepared_y  
0        Bridge Preventive Maintenance - Deck Joints    3  NON-MPO  2018-01-01  
1                              Permanent Restoration    3  NON-MPO  2018-01-01  
2                              Permanent Restoration    5  NON-MPO  2018-01-01  
3  Create A Smart Growth Corridor With Barrier Se...    1    SACOG  2018-01-01  
4                                   Streetscape (tc)    3    SACOG  2018-01-01

In [4]:
locode = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx', sheet_name=None), ignore_index=True)


In [5]:
locode.head()

Agency Locode                                        Agency Name  District  \
0           6302  Humboldt Bay Harbor Recreation & Conservation ...         1   
1           6330           Willow Creek Community Services District         1   
2           5036                                           Trinidad         1   
3           5049                                              Ukiah         1   
4           5082                                            Willits         1   

        County Name                                   RTPA Name MPO Name  \
0   Humboldt County  Humboldt County Association of Governments  NON-MPO   
1   Humboldt County  Humboldt County Association of Governments  NON-MPO   
2   Humboldt County  Humboldt County Association of Governments  NON-MPO   
3  Mendocino County            Mendocino Council of Governments  NON-MPO   
4  Mendocino County            Mendocino Council of Governments  NON-MPO   

  MPO Locode FADS "Active" E76s     (7-12-2021)  
0         NON-MPO                           NaN  
1         NON-MPO                           NaN  
2         NON-MPO                           NaN  
3         NON-MPO                           Yes  
4         NON-MPO                           NaN

In [6]:
locode_df = to_snakecase(locode)

In [7]:
locode_df.rename(columns={'agency_locode':'locode'}, inplace=True)

In [8]:
locode_df.head()

locode                                        agency_name  district  \
0    6302  Humboldt Bay Harbor Recreation & Conservation ...         1   
1    6330           Willow Creek Community Services District         1   
2    5036                                           Trinidad         1   
3    5049                                              Ukiah         1   
4    5082                                            Willits         1   

        county_name                                   rtpa_name mpo_name  \
0   Humboldt County  Humboldt County Association of Governments  NON-MPO   
1   Humboldt County  Humboldt County Association of Governments  NON-MPO   
2   Humboldt County  Humboldt County Association of Governments  NON-MPO   
3  Mendocino County            Mendocino Council of Governments  NON-MPO   
4  Mendocino County            Mendocino Council of Governments  NON-MPO   

  mpo_locode_fads active_e76s______7_12_2021_  
0         NON-MPO                         NaN  
1         NON-MPO                         NaN  
2         NON-MPO                         NaN  
3         NON-MPO                         Yes  
4         NON-MPO                         NaN

In [9]:
locode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041 entries, 0 to 1040
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   locode                       1041 non-null   int64 
 1   agency_name                  1041 non-null   object
 2   district                     1041 non-null   int64 
 3   county_name                  942 non-null    object
 4   rtpa_name                    1041 non-null   object
 5   mpo_name                     1041 non-null   object
 6   mpo_locode_fads              1041 non-null   object
 7   active_e76s______7_12_2021_  464 non-null    object
dtypes: int64(2), object(6)
memory usage: 65.2+ KB


In [10]:
#should be possible based on these two. 

In [11]:
locode_df>>filter(_.agency_name.str.contains('Humboldt'))>>count(_.locode)

locode  n
0    5904  1
1    6133  1
2    6162  1
3    6302  1
4    6487  1

In [12]:
df>>filter(_.agency.str.contains('Humboldt'))>>count(_.locode)

locode    n
0   5904  627
1   5940    1
2   6302    1

In [13]:
locode_df >> filter(_.agency_name.str.contains("Yosemite"))

locode                                       agency_name  district  \
863    6305  Yosemite Area Regional Transportation System JPA        10   
870    6221                            Yosemite National Park        10   

    county_name rtpa_name mpo_name mpo_locode_fads active_e76s______7_12_2021_  
863         NaN  NON-RTPA  NON-MPO         NON-MPO                         Yes  
870         NaN  NON-RTPA  NON-MPO         NON-MPO                         NaN

In [14]:
df>>filter(_.agency==('Yrts'))

prefix project_no agency prepared_date  fed_requested  ac_requested  \
8674     CML  6305(011)   Yrts    2015-07-09        39283.0           0.0   
10563    CML  6305(008)   Yrts    2014-10-13        45717.0           0.0   
11162    CML  6305(010)   Yrts    2014-07-25        85000.0           0.0   
11630    CML  6305(007)   Yrts    2014-06-19            0.0           0.0   

       total_requested status_comment locode  dist  ftip_no  \
8674           39283.0     Authorized    NaN    10  YARTS07   
10563          45717.0     Authorized    NaN    10  YARTS07   
11162          85000.0     Authorized    NaN    10  YARTS07   
11630              0.0     Authorized    NaN    10  YARTS07   

                                 project_location  \
8674              Merced, Mariposa, Mono Counties   
10563  In The Counties, Merced, Mariposa And Mono   
11162             Merced, Mariposa, Mono Counties   
11630               Merced,mariposa,mono Counties   

                                       type_of_work  seq      mpo  prepared_y  
8674       Yarts Public Outreach And Marketing (tc)    1  ER NONE  2015-01-01  
10563  Public Outreach And Marketing For Yarts (tc)    2     SCAG  2014-01-01  
11162      Yarts Public Outreach And Marketing (tc)    1    SJCOG  2014-01-01  
11630                 Public Outreach And Marketing    2  STANCOG  2014-01-01

## Separating out Project Number and Locode


### Separating out the columns

In [15]:
df[["projectNO_locode", "project_stage"]] = df["project_no"].str.split(pat="(", expand=True)

In [16]:
df.projectNO_locode = [x.replace(")", "") for x in df.projectNO_locode]

In [17]:
df.sample()

prefix project_no              agency prepared_date  fed_requested  \
1775     ER  2610(003)  Los Angeles County    2018-04-19            0.0   

      ac_requested  total_requested status_comment locode  dist ftip_no  \
1775           0.0         63062.37     Authorized   5953     7     NaN   

                                       project_location  \
1775  Mulholland Hwy-from Mm 14 To Mm 24; Kanan Rd-f...   

                                           type_of_work  seq   mpo  \
1775  Removed 2,300 Cy Of  Debris & Slide Materials ...    3  SCAG   

      prepared_y projectNO_locode project_stage  
1775  2018-01-01             2610          003)

In [18]:
##dropping projectNO column since we are just interested in ProjectID

df.drop('project_stage', axis=1, inplace=True)

In [19]:
df.sample()

prefix project_no         agency prepared_date  fed_requested  \
6728   HSIP  5942(231)  Fresno County    2016-05-12       526800.0   

      ac_requested  total_requested status_comment locode  dist    ftip_no  \
6728           0.0         902575.0     Authorized   5942     6  FRE040401   

                                      project_location  \
6728  Intersection Of Grantland Avenue And Shaw Avenue   

                                          type_of_work  seq   mpo  prepared_y  \
6728  Traffic Signals, Left-turn Pocket, Road Widening    3  CFCG  2016-01-01   

     projectNO_locode  
6728             5942

In [20]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

In [21]:
df['projectNO_locode'] = df['projectNO_locode'].apply(get_num)

## Create Agency/Locode list from Obligated Data

In [22]:
# creating locode crosswalk from the original data to see if that works. 

In [23]:
group = df.groupby('agency')
crosswalk_all = group.apply(lambda x: x['locode'].unique())
crosswalk_all.head()

agency
Access Services                                 [6312]
Agoura Hills                               [5435, nan]
Ala-Con Costa T                                 [6002]
Alameda                                         [5014]
Alameda - Contra Costa Transit District         [6002]
dtype: object

In [24]:
crosswalk_all >> pipe(_.explode)

<bound method Series.explode of agency
Access Services                                             [6312]
Agoura Hills                                           [5435, nan]
Ala-Con Costa T                                             [6002]
Alameda                                                     [5014]
Alameda - Contra Costa Transit District                     [6002]
                                                    ...           
Yrts                                                         [nan]
Yuba City                                                   [5163]
Yuba County                                                 [5916]
Yucaipa                                    [5457, nan, 5954, NBIL]
Yucca Valley                                           [5466, nan]
Length: 671, dtype: object>

In [25]:
cw = pd.DataFrame([crosswalk_all])


In [26]:
cw

agency Access Services Agoura Hills Ala-Con Costa T Alameda  \
0               [6312]  [5435, nan]          [6002]  [5014]   

agency Alameda - Contra Costa Transit District  \
0                                       [6002]   

agency Alameda Corridor Transportation Authority Alameda County  \
0                                         [6246]         [5933]   

agency Alameda County Congestion Management Agency  \
0                                           [6273]   

agency Alameda County Transportation Commission  \
0                                        [6480]   

agency Alameda County Transportation Improvement Authority  ...  \
0                                                  [6430]   ...   

agency Yolo County Transportation District  Yorba Linda  \
0                                   [6195]  [5402, nan]   

agency Yosemite Area Regional Transportation System Jpa Yountville Yreka City  \
0                                                [6305]     [5395]     [5020]   

agency   Yrts Yuba City Yuba County                  Yucaipa Yucca Valley  
0       [nan]    [5163]      [5916]  [5457, nan, 5954, NBIL]  [5466, nan]  

[1 rows x 671 columns]

In [27]:
cw = cw >> gather('agency', 'locode', _["Access Services":"Yucca Valley"])

In [28]:
cw

agency                   locode
0                            Access Services                   [6312]
1                               Agoura Hills              [5435, nan]
2                            Ala-Con Costa T                   [6002]
3                                    Alameda                   [5014]
4    Alameda - Contra Costa Transit District                   [6002]
..                                       ...                      ...
666                                     Yrts                    [nan]
667                                Yuba City                   [5163]
668                              Yuba County                   [5916]
669                                  Yucaipa  [5457, nan, 5954, NBIL]
670                             Yucca Valley              [5466, nan]

[671 rows x 2 columns]

### Add rows for agencies with more than one locode

In [29]:
cw2 = (cw 
     >> pipe(_.explode('locode'))
) 

In [30]:
cw2

agency locode
0    Access Services   6312
1       Agoura Hills   5435
1       Agoura Hills    NaN
2    Ala-Con Costa T   6002
3            Alameda   5014
..               ...    ...
669          Yucaipa    NaN
669          Yucaipa   5954
669          Yucaipa   NBIL
670     Yucca Valley   5466
670     Yucca Valley    NaN

[885 rows x 2 columns]

In [31]:
cw2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885 entries, 0 to 670
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   agency  885 non-null    object
 1   locode  680 non-null    object
dtypes: object(2)
memory usage: 20.7+ KB


In [32]:
cw2 >> filter(_.locode.isnull())>>count(_.agency)>>arrange(-_.n)

agency  n
0    Agoura Hills  1
1        Alhambra  1
2          Amador  1
3     Amador City  1
4         Anaheim  1
..            ... ..
200      Wildomar  1
201   Yorba Linda  1
202          Yrts  1
203       Yucaipa  1
204  Yucca Valley  1

[205 rows x 2 columns]

### Convert locode to numeric

In [33]:
locode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041 entries, 0 to 1040
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   locode                       1041 non-null   int64 
 1   agency_name                  1041 non-null   object
 2   district                     1041 non-null   int64 
 3   county_name                  942 non-null    object
 4   rtpa_name                    1041 non-null   object
 5   mpo_name                     1041 non-null   object
 6   mpo_locode_fads              1041 non-null   object
 7   active_e76s______7_12_2021_  464 non-null    object
dtypes: int64(2), object(6)
memory usage: 65.2+ KB


In [34]:
#using function get_num from earlier

In [35]:
cw2['locode'] = cw2['locode'].apply(get_num)

## Joining Dataframes

In [36]:
join = (cw2 >> full_join(_, locode_df, on = "locode")) 

In [37]:
join.head()

agency locode      agency_name  district         county_name  \
0  Access Services   6312  Access Services       7.0  Los Angeles County   
1     Agoura Hills   5435     Agoura Hills       7.0  Los Angeles County   
2     Agoura Hills    NaN              NaN       NaN                 NaN   
3         Alhambra    NaN              NaN       NaN                 NaN   
4           Amador    NaN              NaN       NaN                 NaN   

                                           rtpa_name  \
0  Los Angeles County Metropolitan Transportation...   
1                                           NON-RTPA   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                         mpo_name mpo_locode_fads  \
0  Southern California Association Of Governments            SCAG   
1  Southern California Association Of Governments            SCAG   
2                                             NaN             NaN   
3                                             NaN             NaN   
4                                             NaN             NaN   

  active_e76s______7_12_2021_  
0                         NaN  
1                         Yes  
2                         NaN  
3                         NaN  
4                         NaN

## Analysis of the Locodes

### How many have no Locode?

In [38]:
join>>count(_.locode)>>arrange(-_.n)

locode    n
1049    NaN  205
1038   7500   12
538    5953    5
488    5903    4
539    5954    4
...     ...  ...
1042   7504    1
1043   7505    1
1044   32L0    1
1045   38R0    1
1046   38Y0    1

[1050 rows x 2 columns]

In [39]:
join>>filter(_.locode==0)

Empty DataFrame
Columns: [agency, locode, agency_name, district, county_name, rtpa_name, mpo_name, mpo_locode_fads, active_e76s______7_12_2021_]
Index: []

* none that have a locode of 0

### How many have NaN for a Locode?

In [40]:
join>>filter(_.locode.isnull())

agency locode agency_name  district county_name rtpa_name mpo_name  \
2    Agoura Hills    NaN         NaN       NaN         NaN       NaN      NaN   
3        Alhambra    NaN         NaN       NaN         NaN       NaN      NaN   
4          Amador    NaN         NaN       NaN         NaN       NaN      NaN   
5     Amador City    NaN         NaN       NaN         NaN       NaN      NaN   
6         Anaheim    NaN         NaN       NaN         NaN       NaN      NaN   
..            ...    ...         ...       ...         ...       ...      ...   
202      Wildomar    NaN         NaN       NaN         NaN       NaN      NaN   
203   Yorba Linda    NaN         NaN       NaN         NaN       NaN      NaN   
204          Yrts    NaN         NaN       NaN         NaN       NaN      NaN   
205       Yucaipa    NaN         NaN       NaN         NaN       NaN      NaN   
206  Yucca Valley    NaN         NaN       NaN         NaN       NaN      NaN   

    mpo_locode_fads active_e76s______7_12_2021_  
2               NaN                         NaN  
3               NaN                         NaN  
4               NaN                         NaN  
5               NaN                         NaN  
6               NaN                         NaN  
..              ...                         ...  
202             NaN                         NaN  
203             NaN                         NaN  
204             NaN                         NaN  
205             NaN                         NaN  
206             NaN                         NaN  

[205 rows x 9 columns]

In [41]:
print(len(join>>filter(_.locode.isnull())))

205


In [42]:
join>>filter(_.locode.isnull())>>count(_.agency)>>arrange(-_.n)

agency  n
0    Agoura Hills  1
1        Alhambra  1
2          Amador  1
3     Amador City  1
4         Anaheim  1
..            ... ..
200      Wildomar  1
201   Yorba Linda  1
202          Yrts  1
203       Yucaipa  1
204  Yucca Valley  1

[205 rows x 2 columns]

* 205 have null entries

#### Dropping agencies with no locode

In [43]:
join.drop(join[join.locode.isnull()].index, inplace=True)

In [44]:
join>>filter(_.locode.isnull())

Empty DataFrame
Columns: [agency, locode, agency_name, district, county_name, rtpa_name, mpo_name, mpo_locode_fads, active_e76s______7_12_2021_]
Index: []

### What Agencies have incorrect locodes?
Possible from manual entries

In [45]:
join>>count(_.locode)>>arrange(-_.n)>>filter(_.n>1)

locode   n
1038   7500  12
538    5953   5
488    5903   4
539    5954   4
501    5916   3
525    5940   3
542    5957   3
737    6190   3
1047   40A0   3
19     5020   2
273    5275   2
348    5351   2
388    5391   2
460    5463   2
497    5912   2
506    5921   2
515    5930   2
521    5936   2
524    5939   2
541    5956   2
543    5958   2
546    5961   2
549    6000   2
551    6002   2
612    6065   2
628    6081   2
811    6264   2
890    6343   2
912    6365   2
1048   NBIL   2

In [46]:
multiple_id = (join>>count(_.locode)>>arrange(-_.n)>>filter(_.n>1))

In [47]:
print(len(multiple_id))

30


In [48]:
join>>filter(_.locode==5953)

agency locode         agency_name  district  \
526         Los Angeles   5953  Los Angeles County       7.0   
527  Los Angeles County   5953  Los Angeles County       7.0   
528        Marin County   5953  Los Angeles County       7.0   
529            Paradise   5953  Los Angeles County       7.0   
530      Trinity County   5953  Los Angeles County       7.0   

            county_name                                          rtpa_name  \
526  Los Angeles County  Los Angeles County Metropolitan Transportation...   
527  Los Angeles County  Los Angeles County Metropolitan Transportation...   
528  Los Angeles County  Los Angeles County Metropolitan Transportation...   
529  Los Angeles County  Los Angeles County Metropolitan Transportation...   
530  Los Angeles County  Los Angeles County Metropolitan Transportation...   

                                           mpo_name mpo_locode_fads  \
526  Southern California Association Of Governments            SCAG   
527  Southern California Association Of Governments            SCAG   
528  Southern California Association Of Governments            SCAG   
529  Southern California Association Of Governments            SCAG   
530  Southern California Association Of Governments            SCAG   

    active_e76s______7_12_2021_  
526                         Yes  
527                         Yes  
528                         Yes  
529                         Yes  
530                         Yes

In [49]:
(join
    >>filter(_.agency_name.str.contains('Los Angeles County Metropolitan Transportatio'))
     >>select(_.locode, _.agency_name)
)

locode                                        agency_name
486   6065  Los Angeles County Metropolitan Transportation...
487   6065  Los Angeles County Metropolitan Transportation...

In [50]:
@interact
def find_agencies(locode=multiple_id.locode.unique().tolist()):
    
    filtering = (join >> filter(_.locode== locode))
    
    display(Markdown(f"**Agencies using Locode number {locode}**"))
    
    display(filtering)


interactive(children=(Dropdown(description='locode', options=(7500, 5953, 5903, 5954, 5916, 5940, 5957, 6190, …

### Checking the Obligated df for project location

In [51]:
#looking at one of the locodes in the drop down menu
df>>filter(_.locode=='5954')>>filter(_.agency=="Tehama County")

prefix project_no         agency prepared_date  fed_requested  \
19512  ACSTP  40A0(067)  Tehama County    2021-06-03            0.0   

       ac_requested  total_requested status_comment locode  dist ftip_no  \
19512       5478.37          5660.65     Authorized   5954     2     NaN   

                        project_location  \
19512  Dusty Way Closure And Monitoring.   

                                            type_of_work  seq   mpo  \
19512  Ca19-2 Eo Work To Remove Debris And Cleanup.  ...    1  SCAG   

       prepared_y projectNO_locode  
19512  2021-01-01             40A0

In [52]:
locode_df>>filter(_.agency_name=='Tehama County')

locode    agency_name  district    county_name  \
45    5908  Tehama County         2  Tehama County   

                                  rtpa_name mpo_name mpo_locode_fads  \
45  Tehama County Transportation Commission  NON-MPO         NON-MPO   

   active_e76s______7_12_2021_  
45                         Yes

##### ---
* is it worth manually changing the locodes? or not because it might somehow be connected? 

In [53]:
df>>filter(_.locode=='5953')>>filter(_.agency=="Marin County")

prefix project_no        agency prepared_date  fed_requested  \
19568  ACSTP  40A0(087)  Marin County    2021-06-16            0.0   

       ac_requested  total_requested status_comment locode  dist ftip_no  \
19568     676463.01        764107.59     Authorized   5953     4     NaN   

                 project_location  \
19568  Lucas Valley Rd At Mp 3.92   

                                            type_of_work  seq   mpo  \
19568  Lucas Valley Rd At Mp 3.92 Roadway Damage.  Un...    2  SCAG   

       prepared_y projectNO_locode  
19568  2021-01-01             40A0

In [54]:
locode_df >>filter(_.agency_name.str.contains("Marin County"))

locode                       agency_name  district   county_name  \
290    5927                      Marin County         4  Marin County   
320    6343     Marin County Transit District         4  Marin County   
321    6353  Marin County Open Space District         4  Marin County   

                                  rtpa_name  \
290  Metropolitan Transportation Commission   
320  Metropolitan Transportation Commission   
321  Metropolitan Transportation Commission   

                                   mpo_name mpo_locode_fads  \
290  Metropolitan Transportation Commission             MTC   
320  Metropolitan Transportation Commission             MTC   
321  Metropolitan Transportation Commission             MTC   

    active_e76s______7_12_2021_  
290                         Yes  
320                         NaN  
321                         NaN

In [55]:
df>>filter(_.locode=='5953')>>filter(_.agency=="Paradise")


prefix project_no    agency prepared_date  fed_requested  ac_requested  \
16249  ACSTP  38Y0(025)  Paradise    2020-04-06            0.0      107607.5   
17589     ER  38Y0(025)  Paradise    2020-10-29       107607.5     -107607.5   
18969     ER  38Y0(025)  Paradise    2021-05-14            0.0           0.0   
19548     ER  38Y0(025)  Paradise    2021-08-13            0.0           0.0   

       total_requested status_comment locode  dist ftip_no  \
16249         143000.0     Authorized   5953     3     NaN   
17589              0.0     Authorized   5953     3     NaN   
18969              0.0     Authorized   5953     3     NaN   
19548              0.0     Authorized   5953     3     NaN   

                                        project_location  \
16249  In Paradise On Neal Rd From Wayland Rd To Skyway.   
17589  In Paradise On Neal Rd From Wayland Rd To Skyway.   
18969  In Paradise On Neal Rd From Wayland Rd To Skyway.   
19548  In Paradise On Neal Rd From Wayland Rd To Skyway.   

                                    type_of_work  seq   mpo  prepared_y  \
16249  Pavement Repair Of 1.63 Miles Of Neal Rd.    1  SCAG  2020-01-01   
17589  Pavement Repair Of 1.63 Miles Of Neal Rd.    2  SCAG  2020-01-01   
18969  Pavement Repair Of 1.63 Miles Of Neal Rd.    3  SCAG  2021-01-01   
19548  Pavement Repair Of 1.63 Miles Of Neal Rd.    4  SCAG  2021-01-01   

      projectNO_locode  
16249             38Y0  
17589             38Y0  
18969             38Y0  
19548             38Y0

In [56]:
locode_df>>filter(_.locode==5953)

locode         agency_name  district         county_name  \
634    5953  Los Angeles County         7  Los Angeles County   

                                             rtpa_name  \
634  Los Angeles County Metropolitan Transportation...   

                                           mpo_name mpo_locode_fads  \
634  Southern California Association Of Governments            SCAG   

    active_e76s______7_12_2021_  
634                         Yes

In [57]:
locode_df>>filter(_.agency_name=='Paradise')

locode agency_name  district   county_name  \
175    5425    Paradise         3  Butte County   

                                   rtpa_name  \
175  Butte County Association of Governments   

                                    mpo_name mpo_locode_fads  \
175  Butte County Association Of Governments            BCAG   

    active_e76s______7_12_2021_  
175                         Yes

In [58]:
#location is in Paradise, while Locode is Los Angeles County

* Some agencies with same locodes are the same on the oblgiated list, but just have different spellings. 
    * for example: Locode 6002 Alameda-Contra Costa Transit District spelled
        * Ala-Con Consta T 
        * Alameda-Contra Costa Transit District
* Documenting these allows us to have a correct list of agencies instead of having duplicates      
        


## Agencies to Change

| Locodes  |Agency in Locode List | Agencies to Change |
| ------------- | ------------- | ------------- |
| 5953 | Los Angeles County | Los Angeles, Marin County, Paradise, Trinity County  |
| 5903 |Modoc County | Alpine County, Monterey County, Nevada County  |
| 5954 |San Bernardino County | San Bernardino, Tehama County, Yucaipa    |
| 5916 |Yuba County | Shasta County, Tuolumne County|
| 5940 | Mariposa County | Humboldt, Mariposa| 
| 5957 | San Diego County | San Diego, Cathedral City|
| 6190 | U.S. Forest Service, Pacific Southwest Region | Usda Forest Ser, United States Forest Service| 
| 5020 |Yreka City | Sonoma County|
| 5275 | Indio | Palm Springs | 
| 5351 | Pico Rivera | Los Angeles County |
| 5391 | Morro Bay | Ora Co Trans Au | 
| 5463 | Calabasas | Calaveras |
| 5912 | Butte County | Santa Barbara County | 
| 5921 | Napa County | Shasta County |
| 5930 | Calaveras County | Los Angeles County | 
| 5936 | Santa Cruz County | Monterey County |
| 5939 | Merced County | Merced | 
| 5956 | Riverside County | Riverside |
| 5958 | Imperial County | Imperial | 
| 5961 | Kern County (District 9) | Kern |
| 6000 | San Francisco Bay Area Rapid Transit District | Bay Area Rt | 
| 6002 | Alameda - Contra Costa Transit District | Ala-Con Costa T | 
| 6065 | Los Angeles County Metropolitan Transportation... | La Co M T A |
| 6081 | Department of Parks and Recreation | Parks And Rec | 
| 6264 | Santa Clara Valley Transportation Authority | Vta |
| 6343 | Marin County Transit District | Mctd | 
| 6365 | San Francisco Bay Area Water Transit Authority | Wta |

## Agencies to find
  
| Locodes  | Agency in List | Agencies to find |
| ------------- | ------------- | ------------- |
| 7500  | NaN | Banning, Fowler, Lancaster, Los Angeles, Palmdale, Richmond, San Luis Obispo, San Mateo, Sgvc, Stockton, Sutter, Ventura  |
| 40A0  | NaN | Mendocino, San Bernardino, Santa Cruz  |
| NBIL  | NaN | La Quinta, Yucaipa                     |

## Other

### Agencies with multiple locodes

In [59]:
join >> group_by(_.agency) >> summarize(n=_.locode.nunique()) >> arrange(-_.n) >>filter(_.n>2)

agency    n
626                 NaN  421
73             Caltrans   14
276         Los Angeles    4
434      San Bernardino    4
277  Los Angeles County    3
320     Monterey County    3
505       Shasta County    3
624             Yucaipa    3

In [60]:
join>>filter(_.agency=='Caltrans')>>select(_.agency, _.locode, _.agency_name, _.district, _.county_name)>>arrange(_.locode)

agency locode agency_name  district   county_name
301  Caltrans   6200    Caltrans      53.0  Multi-County
308  Caltrans   6201    Caltrans       1.0  Multi-County
299  Caltrans   6202    Caltrans       2.0  Multi-County
303  Caltrans   6203    Caltrans       3.0  Multi-County
300  Caltrans   6204    Caltrans       4.0  Multi-County
305  Caltrans   6205    Caltrans       5.0  Multi-County
302  Caltrans   6206    Caltrans       6.0  Multi-County
297  Caltrans   6207    Caltrans       7.0  Multi-County
306  Caltrans   6208    Caltrans       8.0  Multi-County
309  Caltrans   6209    Caltrans       9.0  Multi-County
307  Caltrans   6210    Caltrans      10.0  Multi-County
304  Caltrans   6211    Caltrans      11.0  Multi-County
298  Caltrans   6212    Caltrans      12.0  Multi-County
310  Caltrans   7504         NaN       NaN           NaN

In [61]:
join>> filter(_.agency=='Los Angeles') >> select(_.agency, _.locode, _.agency_name, _.district, _.county_name) 

agency locode         agency_name  district         county_name
251  Los Angeles   7500                 NaN       NaN                 NaN
525  Los Angeles   5006         Los Angeles       7.0  Los Angeles County
526  Los Angeles   5953  Los Angeles County       7.0  Los Angeles County
531  Los Angeles   38Y0                 NaN       NaN                 NaN

In [62]:
join>> filter(_.agency=='San Bernardino') >> select(_.agency, _.locode, _.agency_name, _.district, _.county_name) 

agency locode            agency_name  district  \
552  San Bernardino   40A0                    NaN       NaN   
691  San Bernardino   5033         San Bernardino       8.0   
692  San Bernardino   5954  San Bernardino County       8.0   
696  San Bernardino   38R0                    NaN       NaN   

               county_name  
552                    NaN  
691  San Bernardino County  
692  San Bernardino County  
696                    NaN

In [63]:
join>> filter(_.agency=='Los Angeles County') >> select(_.agency, _.locode, _.agency_name, _.district, _.county_name) 

agency locode         agency_name  district  \
289  Los Angeles County   5930    Calaveras County      10.0   
527  Los Angeles County   5953  Los Angeles County       7.0   
532  Los Angeles County   5351         Pico Rivera       7.0   

            county_name  
289    Calaveras County  
527  Los Angeles County  
532  Los Angeles County

In [64]:
df>> filter(_.agency=='Los Angeles County') >> filter(_.locode=='5930') 

prefix project_no              agency prepared_date  fed_requested  \
19609  ACSTP  40A0(083)  Los Angeles County    2021-06-23            0.0   

       ac_requested  total_requested status_comment locode  dist ftip_no  \
19609     812174.22         968400.0     Authorized   5930     7     NaN   

                              project_location  \
19609  Mulholland Highway 300' N/o Of Mm 14.74   

                                            type_of_work  seq      mpo  \
19609  Restoration: Construct 85' Enviro Block Gravit...    1  NON-MPO   

       prepared_y projectNO_locode  
19609  2021-01-01             40A0

In [65]:
join>> filter(_.agency=='Monterey County') >> select(_.agency, _.locode, _.agency_name, _.district, _.county_name) 

agency locode        agency_name  district        county_name
220  Monterey County   5903       Modoc County       2.0       Modoc County
575  Monterey County   5944    Monterey County       5.0    Monterey County
576  Monterey County   5936  Santa Cruz County       5.0  Santa Cruz County

In [66]:
join>> filter(_.agency=='Shasta County') >> select(_.agency, _.locode, _.agency_name, _.district, _.county_name) 

agency locode    agency_name  district    county_name
591  Shasta County   5921    Napa County       4.0    Napa County
764  Shasta County   5906  Shasta County       2.0  Shasta County
765  Shasta County   5916    Yuba County       3.0    Yuba County

In [67]:
join>> filter(_.agency=='Yucaipa') >> select(_.agency, _.locode, _.agency_name, _.district, _.county_name) 

agency locode            agency_name  district            county_name
495  Yucaipa   NBIL                    NaN       NaN                    NaN
695  Yucaipa   5954  San Bernardino County       8.0  San Bernardino County
883  Yucaipa   5457                Yucaipa       8.0  San Bernardino County

In [68]:
df>> filter(_.agency=='Yucaipa') >> filter(_.locode=='5954')

prefix project_no   agency prepared_date  fed_requested  ac_requested  \
16672  ACSTP  40A0(062)  Yucaipa    2020-03-19            0.0     324335.57   

       total_requested status_comment locode  dist ftip_no  \
16672         359334.5     Authorized   5954     8     NaN   

                                        project_location  \
16672  Storm Events That Occured In Mid-february, 201...   

                                            type_of_work  seq   mpo  \
16672  Emergency Opening (eo) And Permanent Restorati...    1  SCAG   

       prepared_y projectNO_locode  
16672  2020-01-01             40A0

### Checking project numbers

In [69]:
df>> filter(_.agency=='Los Angeles') >> count(_.project_no)>>arrange(-_.n)

project_no  n
211  5006(839)  9
103  5006(664)  8
191  5006(811)  6
17   5006(170)  5
19   5006(181)  5
..         ... ..
288  5953(653)  1
289  5953(657)  1
290  5953(685)  1
291  7500(284)  1
292  TC03(003)  1

[293 rows x 2 columns]

In [70]:
df>>count(_.project_no)>>arrange(-_.n)

project_no   n
7278  6003(051)  12
7467  6066(121)  10
8188  6328(082)  10
8250  6429(023)  10
1220  5006(839)   9
...         ...  ..
8358  NBIL(546)   1
8359  NBIL(547)   1
8360  NBIS(510)   1
8361  TC03(003)   1
8362  X075(047)   1

[8363 rows x 2 columns]

In [71]:
df>>filter(_.project_no=="6003(051)")

prefix project_no                                             agency  \
1799    BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
3458    BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
5157    BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
8196    BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
13513   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
14613   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
15893   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
18051   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
18276   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
18539   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
19577   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   
19721   BHLS  6003(051)  Golden Gate Bridge, Highway And Transportation...   

      prepared_date  fed_requested  ac_requested  total_requested  \
1799     2018-03-08      9548000.0    -9548000.0              0.0   
3458     2017-06-27      9650000.0    -9650000.0              0.0   
5157     2016-12-28            0.0    76140000.0      115631035.0   
8196     2015-09-15            0.0    49000000.0       76703965.0   
13513    2019-06-06     45239332.0   -45239332.0              0.0   
14613    2019-06-21      8942445.0    -8942445.0              0.0   
15893    2020-01-10      8942441.0    -8942441.0         194868.0   
18051    2020-10-12     14380132.0   -14380132.0        2000000.0   
18276    2021-02-08       619868.0     -619868.0              0.0   
18539    2021-04-19            0.0           0.0              0.0   
19577    2021-06-09     17670668.0    -9760668.0        9563000.0   
19721    2021-10-08      6057114.0    -6057114.0              0.0   

                              status_comment locode  dist    ftip_no  \
1799                              Authorized   6003     4  MRN050019   
3458                              Authorized   6003     4  MRN050019   
5157                              Authorized   6003     4  MRN050019   
8196                              Authorized   6003     4  MRN050019   
13513                             Authorized   6003     4  MRN050019   
14613                             Authorized   6003     4  MRN050019   
15893                             Authorized   6003     4  MRN050019   
18051                             Authorized   6003     4  MRN050019   
18276                             Authorized   6003     4  MRN050019   
18539                             Authorized   6003     4  MRN050019   
19577                             Authorized   6003     4  MRN050019   
19721  ON HOLD - FOR CONSTRAINED BRIDGE LIST   6003     4        NaN   

         project_location              type_of_work  seq  mpo  prepared_y  \
1799   Golden Gate Bridge  Suicide Deterrent System    5  MTC  2018-01-01   
3458   Golden Gate Bridge  Suicide Deterrent System    3  MTC  2017-01-01   
5157   Golden Gate Bridge  Suicide Deterrent System    2  MTC  2016-01-01   
8196   Golden Gate Bridge  Suicide Deterrent System    1  MTC  2015-01-01   
13513  Golden Gate Bridge  Suicide Deterrent System    6  MTC  2019-01-01   
14613  Golden Gate Bridge  Suicide Deterrent System    7  MTC  2019-01-01   
15893  Golden Gate Bridge  Suicide Deterrent System    8  MTC  2020-01-01   
18051  Golden Gate Bridge  Suicide Deterrent System    9  MTC  2020-01-01   
18276  Golden Gate Bridge  Suicide Deterrent System   10  MTC  2021-01-01   
18539  Golden Gate Bridge  Suicide Deterrent System   11  MTC  2021-01-01   
19577  Golden Gate Bridge  Suicide Deterrent System   12  MTC  2021-01-01   
19721  Golden Gate Bridge  Suicide Deterrent System   13  MTC  2021-01-01   

      projectNO_locode  
1799              6003  
3458              6003  
5157              6003  
8196              6003  
13513             6003  
1461

In [72]:
join>>count(_.mpo_locode_fads)

# mpo_name

mpo_locode_fads    n
0            AMBAG   40
1             BCAG   11
2             FCOG   32
3             KCAG   12
4             KCOG   24
5             MCAG   10
6             MCTC    7
7              MTC  188
8          NON-MPO  264
9            SACOG   60
10          SANDAG   47
11           SBCAG   17
12            SCAG  288
13          SHASTA    9
14           SJCOG   17
15          SLOCOG   19
16         STANCOG   13
17           TAHOE    7
18            TCAG   14
19             NaN   22

In [73]:
print(locode_df.mpo_locode_fads.unique())

['NON-MPO' 'SHASTA' 'SACOG' 'BCAG' 'TAHOE' 'MTC' 'SLOCOG' 'AMBAG' 'SBCAG'
 'MCTC' 'FCOG' 'KCAG' 'KCOG' 'TCAG' 'SCAG' 'STANCOG' 'MCAG' 'SJCOG'
 'SANDAG']


In [74]:
locode_df.mpo_locode_fads.nunique()

19

In [75]:
locode_df>>filter(_.agency_name.str.contains("Stanislaus"))

locode                        agency_name  district        county_name  \
853     6089  Stanislaus Council of Governments        10  Stanislaus County   
854     5938                  Stanislaus County        10  Stanislaus County   
867     6253             Stanislaus County Fair        10  Stanislaus County   
1015    6174         Stanislaus National Forest        53                NaN   

                              rtpa_name                           mpo_name  \
853   Stanislaus Council of Governments  Stanislaus Council Of Governments   
854   Stanislaus Council of Governments  Stanislaus Council Of Governments   
867   Stanislaus Council of Governments  Stanislaus Council Of Governments   
1015                           NON-RTPA                            NON-MPO   

     mpo_locode_fads active_e76s______7_12_2021_  
853          STANCOG                         Yes  
854          STANCOG                         Yes  
867          STANCOG                         NaN  
1015         NON-MPO                         NaN